# Audio Recognition

Chegou o momento da QuantumFinance dar voz ao seu atendimento digital. A ideia é permitir que um cliente interaja
por meio de voz também e não somente por texto com a empresa.
Dessa forma, será necessário implementar a vocalização de frases pré-determinadas (Text To Speech), para saudar e
oferecer as opções ao cliente e Speech To Text para identificar a opção selecionada. Mais detalhes abaixo:

## Entrega 1
Gerar as frases para simular um atendimento telefônico, que deverá identificar a empresa, saudar o cliente e
oferecer as opções a seguir: (1) Consulta ao saldo da conta, (2) Simulação de compra internacional, (3) Falar com um
atendente e (4) Sair do atendimento.
As frases devem ser geradas usando alguma das bibliotecas de TTS disponíveis, e armazenadas em arquivo para
serem usadas dentro do loop. Gerar também as frases de resposta correspondentes para confirmar a identificação
de cada opção. Por exemplo: “Você escolheu ‘’Falar com um atendente’’.”

# Entrega 2
Construir um loop que, uma vez iniciado, irá executar os áudios iniciais com as opções. Dentro do loop, o algoritmo
deverá capturar o áudio do usuário usando alguma das bibliotecas de STT e tentar identificar a opção selecionada.
Não é necessário usar nada avançado, uma busca por palavra-chave simples é suficiente.
Uma vez que a opção seja identificada, o áudio de resposta correspondente deverá ser tocado.

## Entrega 3
Caso nenhuma opção tenha sido identificada, um áudio específico informando isso deverá ser executado e as opções
deverão ser repetidas.

## Entrega 4
O loop deverá ser encerrado quando a opção “Sair do atendimento” for selecionada.

In [1]:
!pip install ffmpeg-python

In [2]:
!pip install SpeechRecognition==3.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 31.1 MB/s eta 0:00:00


In [3]:
!pip install gtts

In [4]:
import speech_recognition as sr

In [5]:
from IPython.display import HTML, Audio,display
import ffmpeg
from time import sleep

In [6]:
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
from scipy.io.wavfile import write
from gtts import gTTS,lang
from numpy import lib
import re

In [7]:

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.mimeType = 'audio/wav';
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Clique para parar a gravação..";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Processando o áudio... por favor aguarde.."
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

In [8]:
# Saudação inicial
saudacoes = "Bem-vindo ao canal de atendimento da empresa chamada FIAP. Como posso ajudar você?"

# Frases para oferecer as opções
option_prompts = [
    "Para consulta ao saldo da conta, escolha opção 1.",
    "Para simulação de compra internacional, escolha opção 2.",
    "Para falar com um atendente, escolha opção 3.",
    "Sair do atendimento, escolha opção 4."
]

# Responses com cada resposta
responses = [
     "Você escolheu 'Consulta ao saldo da conta'.",
    "Você escolheu 'Simulação de compra internacional'.",
    "Você escolheu 'Falar com um atendente'.",
    "Você escolheu 'Sair do atendimento'."
]

# Função para salvar áudio do microfone em um arquivo
def save_audio_microphone(audio, filename):
    # Escala o áudio para int16 antes de salvar
    scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
    # Salva o áudio no formato WAV
    write(filename, 44100, scaled)

# Função para realizar a transcrição de fala usando o Google Speech Recognition
def stt_google(filename, mylang):
    recog = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = recog.record(source)
        rec_text = recog.recognize_google(audio, language=mylang)
        return rec_text

# Função para obter áudio do usuário, realizar a transcrição e retornar o texto
def get_audio(lang):
    # Exibe um botão no ambiente interativo para iniciar a gravação de áudio
    display(HTML(AUDIO_HTML))

    # Obtém os dados de áudio do ambiente interativo
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])

    # Processa os dados de áudio usando a biblioteca ffmpeg
    process = (ffmpeg
        .input('pipe:0')
        .output('pipe:1', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    # Ajusta o cabeçalho WAV para garantir compatibilidade
    riff_chunk_size = len(output) - 8
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)
    riff = output[:4] + bytes(b) + output[8:]

    # Lê o áudio do arquivo WAV e salva em um arquivo local
    s, audio = wav_read(io.BytesIO(riff))
    filename = 'audio_microphone.wav'
    save_audio_microphone(audio, filename)

    # Realiza a transcrição de acordo com a biblioteca escolhida
    texto = "None"
    texto = stt_google(filename, lang)
    return texto

In [9]:
def extrair(text):
    numeros = re.findall(r'\d+', text)
    number_string = ''.join(numeros)
    return number_string


In [12]:
#Inicializa a variável de controle do loop
loop_break = 0

#Loop principal
while loop_break == 0:
    #Gera a saudação inicial e a reproduz
    tts = gTTS(saudacoes, lang='pt')
    tts.save('bemvindo.wav')
    sound_file = 'bemvindo.wav'
    wn = Audio(sound_file, autoplay=True)
    display(wn)
    sleep(7)

    #Loop para oferecer as opções
    for mensagens in range(len(option_prompts)):
        #Gera a mensagem da opção atual e a reproduz
        tts = gTTS(option_prompts[mensagens], lang='pt')
        tts.save('opcao.wav')
        sound_file = 'opcao.wav'
        wn = Audio(sound_file, autoplay=True)
        sleep(6)
        display(wn)

    try:
        #Aguarda 2 segundos antes de obter a opção escolhida pelo usuário através do áudio
        sleep(2)
        opcao_escolhida = get_audio("pt-BR")
        opcao_escolhida_n = extrair(opcao_escolhida)
        sleep(2)

        #Gera a mensagem correspondente à opção escolhida e a reproduz
        tts = gTTS(responses[int(opcao_escolhida_n)-1], lang='pt')
        tts.save('opcao_escolhida.wav')
        sound_file = 'opcao_escolhida.wav'
        wn = Audio(sound_file, autoplay=True)
        display(wn)
        sleep(6)
        #Verifica se a opção escolhida é a de sair (opção 4)
        if int(opcao_escolhida_n) == 4:
            saida = "Você saiu do canal de atendimento. Tenha um bom dia."
            tts = gTTS(saida, lang='pt')
            tts.save('saida.wav')
            sound_file = 'saida.wav'
            wn = Audio(sound_file, autoplay=True)
            display(wn)
            loop_break = -1  # Encerra o loop

    except sr.UnknownValueError:
        #Trata o caso em que nenhuma opção é identificada
        recusa = "Nenhuma opção identificada."
        tts = gTTS(recusa, lang='pt')
        tts.save('recusa.wav')
        sound_file = 'recusa.wav'
        wn = Audio(sound_file, autoplay=True)
        display(wn)

    except sr.RequestError as e:
        #Trata o caso de erro na requisição do serviço de reconhecimento de fala
        recusa = "Erro na requisição do serviço de reconhecimento de fala."
        tts = gTTS(recusa, lang='pt')
        tts.save('recusa.wav')
        sound_file = 'recusa.wav'
        wn = Audio(sound_file, autoplay=True)
        display(wn)

    #Aguarda 10 segundos antes de reiniciar o loop
    sleep(10)

KeyboardInterrupt: 